<a href="https://colab.research.google.com/github/wooheehee/deeplearning-practice/blob/main/CH4_2_MNIST_%EC%88%AB%EC%9E%90%EB%B6%84%EB%A5%98%EA%B8%B0_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
x_train, x_test = x_train.reshape([-1,784]), x_test.reshape([-1,784])
x_train, x_test = x_train / 255., x_test / 255.
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

11490434/11490434 [==============================] - 0s 0us/step


In [2]:
learning_rate = 0.001
num_epochs = 30
batch_size = 256
display_step = 1
input_size = 784
hidden1_size = 256
hidden2_size = 256
output_size = 10

In [4]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size)

In [14]:
def random_normal_initializer_with_stddev_1():
  return tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.0, seed=None)

In [15]:
class ANN(tf.keras.Model):
  def __init__(self):
    super(ANN, self).__init__()
    self.hidden_layer_1 = tf.keras.layers.Dense(hidden1_size,
                                                activation='relu',
                                                kernel_initializer=random_normal_initializer_with_stddev_1(),
                                                bias_initializer=random_normal_initializer_with_stddev_1())
    self.hidden_layer_2 = tf.keras.layers.Dense(hidden2_size,
                                                activation='relu',
                                                kernel_initializer=random_normal_initializer_with_stddev_1(),
                                                bias_initializer=random_normal_initializer_with_stddev_1())
    self.output_layer = tf.keras.layers.Dense(output_size,
                                              activation=None,
                                              kernel_initializer=random_normal_initializer_with_stddev_1(),
                                              bias_initializer=random_normal_initializer_with_stddev_1())
    
  def call(self, x):
    H1_output = self.hidden_layer_1(x)
    H2_output = self.hidden_layer_2(H1_output)
    logits = self.output_layer(H2_output)

    return logits

In [8]:
@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

In [9]:
optimizer = tf.optimizers.Adam(learning_rate)

In [11]:
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = cross_entropy_loss(y_pred, y)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [12]:
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

In [16]:
ANN_model = ANN()

In [18]:
for epoch in range(num_epochs):
  average_loss = 0.
  total_batch = int(x_train.shape[0] / batch_size)

  for batch_x, batch_y in train_data:
    _, current_loss = train_step(ANN_model, batch_x, batch_y), cross_entropy_loss(ANN_model(batch_x), batch_y)
    average_loss += current_loss / total_batch

  if epoch % display_step == 0:
    print("반복(Epoch): %d, 손실 함수(Loss): %f" % ((epoch+1), average_loss))

  print("정확도(Accuracy): %f" % compute_accuracy(ANN_model(x_test), y_test))

반복(Epoch): 1, 손실 함수(Loss): 2.663569
정확도(Accuracy): 0.939000
반복(Epoch): 2, 손실 함수(Loss): 2.139293
정확도(Accuracy): 0.940800
반복(Epoch): 3, 손실 함수(Loss): 1.650359
정확도(Accuracy): 0.941500
반복(Epoch): 4, 손실 함수(Loss): 1.312376
정확도(Accuracy): 0.940300
반복(Epoch): 5, 손실 함수(Loss): 1.040895
정확도(Accuracy): 0.939700
반복(Epoch): 6, 손실 함수(Loss): 0.817604
정확도(Accuracy): 0.941000
반복(Epoch): 7, 손실 함수(Loss): 0.602507
정확도(Accuracy): 0.942300
반복(Epoch): 8, 손실 함수(Loss): 0.456033
정확도(Accuracy): 0.942800
반복(Epoch): 9, 손실 함수(Loss): 0.370694
정확도(Accuracy): 0.942800
반복(Epoch): 10, 손실 함수(Loss): 0.289971
정확도(Accuracy): 0.942200
반복(Epoch): 11, 손실 함수(Loss): 0.228113
정확도(Accuracy): 0.941800
반복(Epoch): 12, 손실 함수(Loss): 0.185002
정확도(Accuracy): 0.944400
반복(Epoch): 13, 손실 함수(Loss): 0.125430
정확도(Accuracy): 0.943600
반복(Epoch): 14, 손실 함수(Loss): 0.083599
정확도(Accuracy): 0.944500
반복(Epoch): 15, 손실 함수(Loss): 0.098090
정확도(Accuracy): 0.947400
반복(Epoch): 16, 손실 함수(Loss): 0.097499
정확도(Accuracy): 0.948200
반복(Epoch): 17, 손실 함수(Loss): 0.055